In [13]:
import pandas as pd
import sqlite3
import uuid
import time
import random

In [14]:
PATH_TO_DATABASE = "C:\\Users\\tora2\\IdeaProjects\\cityScope\\data\\database\\yokosuka_test.db"

In [24]:
conn = sqlite3.connect(PATH_TO_DATABASE)
cursor = conn.cursor()
cursor.execute("SELECT * FROM poi_intervened")
for r in cursor.fetchall():
    print(r)
conn.close()

(1728259664819, 'be24202d-ec36-4e71-b91c-50c50fe567c6', None, None, 35.35104166666667, 139.1328125, None, '53391418', None, None, None, 1)
(1728259675871, 'be24202d-ec36-4e71-b91c-50c50fe567c6', None, None, 35.35104166666667, 139.1328125, None, '53391415', None, None, None, 0)


In [25]:
conn = sqlite3.connect(PATH_TO_DATABASE)
cursor = conn.cursor()
cursor.execute("SELECT * FROM read_progress")
for r in cursor.fetchall():
    print(r)
conn.close()

(1728259675871,)


In [17]:
# テーブル初期化
conn = sqlite3.connect(PATH_TO_DATABASE)
cursor = conn.cursor()
cursor.execute("""DROP TABLE IF EXISTS poi_intervened""")
cursor.execute("""CREATE TABLE poi_intervened(
                time_stamp INTEGER ,
                poi_id TEXT, 
                genre TEXT, 
                area REAL, 
                latitude REAL, 
                longitude REAL, 
                address TEXT, 
                mesh_code TEXT, 
                industry_type TEXT, 
                behavior_type TEXT, 
                abs_attract_score INTEGER,
                is_generated INTEGER
                )""")
cursor.execute("""DROP TABLE IF EXISTS read_progress""")
cursor.execute("""CREATE TABLE read_progress(
                time_stamp INTEGER
                )""")
conn.commit()
conn.close()

In [18]:
def mock_intervened(poi_id=None, genre=None, area=None, latitude=None, longitude=None,
                    address=None, mesh_code=None, industry_type=None, behavior_type=None, is_generated=True):
    # デフォルト値の設定
    if poi_id is None:
        poi_id = str(uuid.uuid4())  # uuidを自動生成

    # 現在のUNIXエポックからの経過時間をミリ秒単位で取得
    time_stamp = int(time.time() * 1000)  # UNIXタイムスタンプをミリ秒に変換
    
    # sqlite用のbooleanに変換
    is_generated_for_sql = 1 if is_generated else 0
    

    # SQLiteデータベースに接続
    conn = sqlite3.connect(PATH_TO_DATABASE)
    cursor = conn.cursor()

    # モック用の挿入クエリ
    insert_query = """
    INSERT INTO poi_intervened (time_stamp, poi_id, genre, area, latitude, longitude, address, mesh_code, industry_type, behavior_type, is_generated)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
    """

    # バインドするパラメータ
    parameters = (
        time_stamp,    # UNIXエポックのミリ秒
        poi_id,
        genre,
        area,
        latitude,
        longitude,
        address,
        mesh_code,
        industry_type,
        behavior_type,
        is_generated_for_sql
    )
    
    # クエリにパラメータをバインドして実行
    cursor.execute(insert_query, parameters)

    # コミットして接続を閉じる
    conn.commit()
    conn.close()

    # 実行されたSQLクエリの内容を確認用に表示
    formatted_query = insert_query.replace('?', '{}').format(*parameters)
    print("Executed SQL query:")
    print(formatted_query)

def mesh_code_to_center_lat_lon(mesh_code):
    # メッシュコードの文字列が正しい長さかチェック
    if len(mesh_code) < 7:
        raise ValueError("メッシュコードが短すぎます。7桁以上である必要があります。")

    # メッシュコードの各桁を取得
    mesh1_lat = int(mesh_code[:2])  # 1次メッシュ緯度
    mesh1_lon = int(mesh_code[2:4])  # 1次メッシュ経度
    mesh2 = int(mesh_code[4])  # 2次メッシュ
    mesh3 = int(mesh_code[5])  # 3次メッシュ
    mesh4 = int(mesh_code[6])  # 250mメッシュ

    # 基本座標を設定（1次メッシュ単位の大きさ）
    lat_base = mesh1_lat * 2 / 3  # 1次メッシュは緯度80km（約0.66666度）
    lon_base = mesh1_lon + 100  # 経度は東経100度基準

    # 2次メッシュ（10km単位）の緯度経度オフセット
    lat_offset_2 = (mesh2 // 2) * (1 / 12)  # 10kmメッシュ = 0.08333度 (1/12)
    lon_offset_2 = (mesh2 % 2) * (1 / 8)    # 10kmメッシュ = 0.125度 (1/8)

    # 3次メッシュ（1km単位）の緯度経度オフセット
    lat_offset_3 = (mesh3 // 2) * (1 / 120)  # 1kmメッシュ = 0.008333度 (1/120)
    lon_offset_3 = (mesh3 % 2) * (1 / 80)    # 1kmメッシュ = 0.0125度 (1/80)

    # 250mメッシュのオフセット
    lat_offset_4 = (mesh4 // 2) * (1 / 240)  # 250mメッシュ = 0.004166度 (1/240)
    lon_offset_4 = (mesh4 % 2) * (1 / 160)   # 250mメッシュ = 0.00625度 (1/160)

    # メッシュ中心の緯度経度計算
    lat_center = lat_base + lat_offset_2 + lat_offset_3 + lat_offset_4 + (1 / 480) / 2  # メッシュ中心のため、半分のサイズを追加
    lon_center = lon_base + lon_offset_2 + lon_offset_3 + lon_offset_4 + (1 / 320) / 2

    return lat_center, lon_center

In [19]:
target_mesh = ['53391415', '53391416', '53391417', '53391418',
               '53391405', '53391406', '53391407', '53391408',
               '53390495', '53390496', '53390497', '53390498',
               '53390485', '53390486', '53390487', '53390488']
genre = "3001"

In [23]:
chosen_mesh = random.choice(target_mesh)
lat, lon = mesh_code_to_center_lat_lon(chosen_mesh)
mock_intervened(poi_id="be24202d-ec36-4e71-b91c-50c50fe567c6", mesh_code=chosen_mesh, latitude=lat, longitude=lon, is_generated=False)

Executed SQL query:

    INSERT INTO poi_intervened (time_stamp, poi_id, genre, area, latitude, longitude, address, mesh_code, industry_type, behavior_type, is_generated)
    VALUES (1728259675871, be24202d-ec36-4e71-b91c-50c50fe567c6, None, None, 35.35104166666667, 139.1328125, None, 53391415, None, None, 0);
    
